In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp chunkadelic

# chunkadelic


> Console script and callable function for preprocessing dataset of disparate-sized audio files into uniform chunks

Note: Duplicates the directory structure(s) referenced by input paths. 

In [ ]:
#|hide
from nbdev.showdoc import *

In [ ]:
#|export
import argparse 
import os 
from multiprocessing import Pool, cpu_count, Barrier
from functools import partial
from tqdm.contrib.concurrent import process_map  
import torch
import torchaudio
import math
from aeiou.core import is_silence, load_audio, makedir, get_audio_filenames, normalize_audio, get_dbmax

In [ ]:
#|export
def blow_chunks(
    audio:torch.tensor,  # long audio file to be chunked
    new_filename:str,    # stem of new filename(s) to be output as chunks
    chunk_size:int,      # how big each audio chunk is, in samples
    sr=48000,            # audio sample rate in Hz
    norm='False',      # normalize input audio, based on the max of the absolute value ['global','channel', or anything else for None, e.g. False]
    spacing=0.5,         # fraction of each chunk to advance between hops
    strip=False,    # strip silence: chunks with max power in dB below this value will not be saved to files
    thresh=-70      # threshold in dB for determining what counts as silence 
    ):
    "chunks up the audio and saves them with --{i} on the end of each chunk filename"
    chunk = torch.zeros(audio.shape[0], chunk_size)
    _, ext = os.path.splitext(new_filename)
    
    # normalize audio if requested 
    #SHH if norm is True: # handle the most likely improper response defaulted to 'global'
    #SHH     norm = 'global'
    if norm in ['global','channel']:       
        audio = normalize_audio(audio, norm)     
        #SHH gain_db = abs(get_dbmax(audio)) - abs(get_dbmax(audio_norm)) 
        #SHH print("")
        #SHH print(f"normalized {new_filename} with type {norm} creating {gain_db} dB change ", flush=True)
        #SHH audio=audio_norm

    spacing = 0.5 if spacing == 0 else spacing # handle degenerate case as a request for the defaults
    
    start, i = 0, 0
    while start < audio.shape[-1]:
        out_filename = new_filename.replace(ext, f'--{i}'+ext) 
        end = min(start + chunk_size, audio.shape[-1])
        if end-start < chunk_size:  # needs zero padding on end
            chunk = torch.zeros(audio.shape[0], chunk_size)
        chunk[:,0:end-start] = audio[:,start:end]
        if (not strip) or (not is_silence(chunk, thresh=thresh)):
            torchaudio.save(out_filename, chunk, sr)
        else:
            print(f"skipping chunk {out_filename} because it's 'silent' (below threhold of {thresh} dB).",flush=True)
        start, i = start + int(spacing * chunk_size), i + 1
    return 

In [ ]:
#|export 
def process_one_file(
    filenames:list,      # list of filenames from which we'll pick one
    file_ind,            # index of file in filenames list
    output_path='test_chunks',
    input_paths=['examples/'], 
    chunk_size=131072, 
    norm='False',       # anything other than 'global' or 'channel' will yield no normalization 
    spacing=0.5, 
    sr=48000, 
    strip=False, 
    thresh=-70, 
    nomix=False,         # exclude 'Mix' outputs from Belmont BDCT dataset
    ):
    "this chunks up one file"
    filename = filenames[file_ind]  # this is actually input_path+/+filename
    new_filename = None
    
    for ipath in input_paths: # set up the output filename & any folders it needs
        if nomix and ('Mix' in ipath) and ('Audio Files' in ipath): return  # this is specific to the BDCT dataset, otherwise ignore
        if ipath in filename:
            last_ipath = ipath.split('/')[-1]           # get the last part of ipath
            clean_filename = filename.replace(ipath,'') # remove all of ipath from the front of filename
            new_filename = f"{output_path}/{last_ipath}/{clean_filename}".replace('//','/') 
            makedir(os.path.dirname(new_filename))      # we might need to make a directory for the output file
            break
    
    if new_filename is None:
        print(f"ERROR: Something went wrong with name of input file {filename}. Skipping.",flush=True) 
        return 
    #try:
    audio = load_audio(filename, sr=sr)
    blow_chunks(audio, new_filename, chunk_size, sr=sr, norm=norm, spacing=spacing, strip=strip, thresh=thresh)
    #except Exception as e: 
    #    print(f"Error '{e}' while loading {filename} or writing chunks. Skipping.", flush=True)

    return

In [ ]:
#|export
def chunkadelic(
    # see argparse setup for meanings of all vars
    output_path='test_chunk',
    input_paths=['examples/'], 
    chunk_size=131072, 
    norm='', 
    spacing=0.5, 
    sr=48000, 
    strip=False, 
    thresh=-70, 
    workers=14,
    nomix=False, 
    verbose=True, # whether to include print statements or not showing progress
    ):
    "function version of main CLI; multiprocessing wrapper calls process_one_file"
    if verbose:
        print("chunkadelic:")
        print(f"   output_path = {output_path}")
        print(f"   input_paths = {input_paths}")
        print(f"   chunk_size  = {chunk_size}")
        print(f"   norm        = {norm}")
        print(f"   spacing     = {spacing}")
        print(f"   sr          = {sr}")
        print(f"   strip       = {strip}")
        print(f"   thresh      = {thresh}")
        print(f"   workers     = {workers}")
        print(f"   nomix       = {nomix}")
        print("Getting list of input filenames")
    filenames = get_audio_filenames(input_paths) 
    n = len(filenames)   
    if verbose:
        print(f"  Got {n} input filenames") 
        print("Processing files (in parallel)")
        if not (norm in ['global','channel']): 
            print(f"Warning: since norm = {norm}, no normalizations will be performed.")
            
    wrapper = partial(process_one_file, filenames, 
                      output_path=output_path, input_paths=input_paths, chunk_size=chunk_size,
                      norm=norm, spacing=spacing, sr=sr, strip=strip, thresh=thresh, nomix=nomix, )
    r = process_map(wrapper, range(0, n), chunksize=1, max_workers=workers)  # chunksize is unfortunate coincidental name used by tqdm, nothing to do with audio chunk size

    if verbose: print("Finished")    

Testing `chunkadelic` function:

In [ ]:
chunkadelic(norm='global', verbose=True)

chunkadelic:
   output_path = test_chunk
   input_paths = ['examples/']
   chunk_size  = 131072
   norm        = global
   spacing     = 0.5
   sr          = 48000
   strip       = False
   thresh      = -70
   workers     = 14
   nomix       = False
Getting list of input filenames
  Got 2 input filenames
Processing files (in parallel)


  0%|          | 0/2 [00:00<?, ?it/s]

Resampling examples/example.wav from 44100 Hz to 48000 Hz
Finished


In [ ]:
#|export
def main():
    parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    parser.add_argument('--chunk_size', type=int, default=2**17, help='Length of chunks')
    parser.add_argument('--sr', type=int, default=48000, help='Output sample rate')
    parser.add_argument('--norm', default='False', const='False', nargs='?', choices=['False', 'global', 'channel'],
                   help='Normalize audio, based on the max of the absolute value [global/channel/False]')
    parser.add_argument('--spacing', type=float, default=0.5, help='Spacing factor, advance this fraction of a chunk per copy')
    parser.add_argument('--strip', action='store_true', help='Strips silence: chunks with max dB below <thresh> are not outputted')
    parser.add_argument('--thresh', type=int, default=-70, help='threshold in dB for determining what constitutes silence')
    parser.add_argument('--workers', type=int, default=min(32, os.cpu_count() + 4), help='Maximum number of workers to use (default: all)')
    parser.add_argument('--nomix', action='store_true',  help='(BDCT Dataset specific) exclude output of "*/Audio Files/*Mix*"')
    parser.add_argument('output_path', help='Path of output for chunkified data')
    parser.add_argument('input_paths', nargs='+', help='Path(s) of a file or a folder of files. (recursive)')
    #args = parser.parse_args()
    args = {k:v for k,v in vars(parser.parse_args()).items() if v is not None} # convert to dict
    
    chunkadelic(**args, verbose=True)

In [ ]:
#| hide
from nbdev import nbdev_export
nbdev_export()

In [ ]:
! chunkadelic -h

usage: chunkadelic [-h] [--chunk_size CHUNK_SIZE] [--sr SR]
                   [--norm [{False,global,channel}]] [--spacing SPACING]
                   [--strip] [--thresh THRESH] [--workers WORKERS] [--nomix]
                   output_path input_paths [input_paths ...]

positional arguments:
  output_path           Path of output for chunkified data
  input_paths           Path(s) of a file or a folder of files. (recursive)

optional arguments:
  -h, --help            show this help message and exit
  --chunk_size CHUNK_SIZE
                        Length of chunks (default: 131072)
  --sr SR               Output sample rate (default: 48000)
  --norm [{False,global,channel}]
                        Normalize audio, based on the max of the absolute
                        value [global/channel/False] (default: False)
  --spacing SPACING     Spacing factor, advance this fraction of a chunk per
                        copy (default: 0.5)
  --strip               Strips silence: chunks wit


---
Testing of CLI run:

In [ ]:
import subprocess

In [ ]:
for norm in ['False', 'global','channel']:
    print("-----\n norm =",norm)
    result = subprocess.run(['chunkadelic', '--norm', norm, 'test_chunks','examples/'], stdout=subprocess.PIPE)
    out = result.stdout.decode("utf-8") 
    print(out)
    assert 'error' not in out.lower(), f'Error occured while running with norm={norm}'  # for CI testing

-----
 norm = False


100%|██████████| 2/2 [00:03<00:00,  1.79s/it]


chunkadelic:
   output_path = test_chunks
   input_paths = ['examples/']
   chunk_size  = 131072
   norm        = False
   spacing     = 0.5
   sr          = 48000
   strip       = False
   thresh      = -70
   workers     = 14
   nomix       = False
Getting list of input filenames
  Got 2 input filenames
Processing files (in parallel)
Resampling examples/example.wav from 44100 Hz to 48000 Hz
Finished

-----
 norm = global


100%|██████████| 2/2 [00:03<00:00,  1.80s/it]


chunkadelic:
   output_path = test_chunks
   input_paths = ['examples/']
   chunk_size  = 131072
   norm        = global
   spacing     = 0.5
   sr          = 48000
   strip       = False
   thresh      = -70
   workers     = 14
   nomix       = False
Getting list of input filenames
  Got 2 input filenames
Processing files (in parallel)
Resampling examples/example.wav from 44100 Hz to 48000 Hz
Finished

-----
 norm = channel


100%|██████████| 2/2 [00:03<00:00,  1.80s/it]


chunkadelic:
   output_path = test_chunks
   input_paths = ['examples/']
   chunk_size  = 131072
   norm        = channel
   spacing     = 0.5
   sr          = 48000
   strip       = False
   thresh      = -70
   workers     = 14
   nomix       = False
Getting list of input filenames
  Got 2 input filenames
Processing files (in parallel)
Resampling examples/example.wav from 44100 Hz to 48000 Hz
Finished

